In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
from sqlalchemy import create_engine
from shapely.geometry import shape
from io import BytesIO
import base64

from dotenv import load_dotenv
import os
import json

In [4]:
# .env 파일을 찾아 환경 변수로 로드
load_dotenv()

# 환경 변수 가져오기
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

# ▶️ MySQL 연결

In [5]:



engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{database}")


def get_animal_hospital_registry():
  query = """
  SELECT jibun_address
  FROM animal_hospital_registry
  WHERE (status_name LIKE '영업/정상') AND (jibun_address LIKE '서울특별시%%')
  """
  df = pd.read_sql(query, engine)

  # 구 이름 추출 방법 1: 문자열 분할
  df['district'] = df['jibun_address'].str.split(' ').str[2]

  # 구 이름 추출 방법 2: 정규표현식 사용
  import re
  df['district'] = df['jibun_address'].str.extract(r'서울특별시 ([가-힣]+구)')

  # 구별 개수 집계
  district_counts = df['district'].value_counts().reset_index()
  district_counts.columns = ['district', 'hospital_count']
  district_counts = district_counts.sort_values('hospital_count', ascending=False)

  return district_counts

get_animal_hospital_registry()

,district,hospital_count
0,강남구,94
1,송파구,83
2,서초구,53
3,강서구,50
4,양천구,49
5,마포구,47
6,강동구,47
7,노원구,45
8,은평구,42
9,영등포구,37
